<a href="https://colab.research.google.com/github/iamsanthoshg/colabFiles/blob/main/C_R_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install langchain langchain_groq langchain_community chromadb huggingface_hub

In [3]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 823.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [19]:
# Import necessary libraries
import os
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_groq import ChatGroq
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory

In [5]:
# 🔹 Step 1: Set Groq API Key
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual API key

In [6]:
# 🔹 Step 2: Load Hugging Face Embeddings
print("Loading Hugging Face embeddings...")
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
print("Embedding model loaded successfully.\n")

Loading Hugging Face embeddings...


<ipython-input-6-7917172477ed>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warning

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded successfully.



In [21]:
# 🔹 Step 3: Load and Process Documents
print("Loading document...")
loader = TextLoader("speech.txt")  # Make sure speech.txt is in the same directory
documents = loader.load()
print(f"Document loaded successfully: {len(documents)} document(s) found.\n")

# 🔹 Step 4: Split Documents into Chunks
print("Splitting document into smaller chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print(f"Document splitting completed: {len(docs)} chunks created.\n")

# 🔹 Step 5: Create Vector Database (Chroma)
print("Creating vector database with ChromaDB...")
vector_db = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./chroma_db")
retriever = vector_db.as_retriever(search_kwargs={"k": 2})  # Retrieve top 5 matches
print("Vector database created and persisted successfully.\n")

# 🔹 Step 6: Load LLM (Groq API)
print("Loading Mistral model from Groq API...")
llm = ChatGroq(model_name="llama3-8b-8192", groq_api_key=os.environ["GROQ_API_KEY"])
print("Mistral model loaded successfully.\n")

# 🔹 Step 7: Add Conversational Memory
print("Initializing conversational memory...")
# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
print("Memory initialized successfully.\n")

# 🔹 Step 8: Create Conversational Retrieval Chain
print("Initializing Conversational Retrieval Chain...")
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    output_key="answer",
    verbose=True
)
print("Conversational Retrieval Chain initialized successfully.\n")

# 🔹 Step 9: Run Interactive Chat Loop
print("\n===== Chatbot Ready! Start asking questions =====")
while True:
    query = input("\nYou: ")

    if query.lower() in ["exit", "quit", "stop"]:
        print("Exiting chatbot. Goodbye!")
        break

    response = conversation_chain.invoke({"question": query})

    # Display Answer
    print("\n🤖 Bot:", response["answer"])

    chat_history = memory.load_memory_variables({})
    print("\n📝 Chat History:", chat_history["chat_history"])


Loading document...
Document loaded successfully: 1 document(s) found.

Splitting document into smaller chunks...
Document splitting completed: 10 chunks created.

Creating vector database with ChromaDB...
Vector database created and persisted successfully.

Loading Mistral model from Groq API...
Mistral model loaded successfully.

Initializing conversational memory...
Memory initialized successfully.

Initializing Conversational Retrieval Chain...
Conversational Retrieval Chain initialized successfully.


===== Chatbot Ready! Start asking questions =====

You: hi


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearan